<a href="https://colab.research.google.com/github/yuvi-s64/YuviN-DataScience-GenAI-Submissions/blob/main/2_02_Joining_DataFrames_COMPLETED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 2.02 Pandas: Joining and merging dataframes
### Why Combine Dataframes?
Quite often in practical applications we have data coming from multiple sources such as different files, database tables or IoT devices. This is particularly likely when working in data science and artificial intelligence. However, typically we want to create a single dataframe to run our analysis and/or models. To achieve this commonly we would want to combine dataframes together.

Joining data is a large topic and an important one in database administration and data engineering. We will look at this at a relatively high level, althout the interested reader can certainly explore this in much more depth.

Let us begin by using the orders_df from the previous session.

In [10]:
import pandas as pd
import numpy as np

orders = {'o10001':{'date':'2024/01/10', 'product':'Hoodie', 'quantity':'1'},
            'o10002':{'date':'2024/01/13', 'product':'Tote bag', 'quantity':'2'},
            'o10003':{'date':'2024/01/14', 'product':'Pencil', 'quantity':'10'},
            'o10004':{'date':'2024/01/15', 'product':'T-shirt', 'quantity':'2'}
}

orders_df = pd.DataFrame(orders)
orders_df

,o10001,o10002,o10003,o10004
date,2024/01/10,2024/01/13,2024/01/14,2024/01/15
product,Hoodie,Tote bag,Pencil,T-shirt
quantity,1,2,10,2


In this scenario, we also have some data about these products from our operations team. The data is currently in a dictionary and can be transformed in the usual fashion:

In [2]:
products = {'123':{'name':'Hoodie', 'cost_price':12.12, 'sale_price':15.00},
            '124':{'name':'Tote bag', 'cost_price':2.15, 'sale_price':9.99},
            '125':{'name':'Pencil', 'cost_price':22.45, 'sale_price':49.99},
            '126':{'name':'T-shirt', 'cost_price':0.45, 'sale_price':2.99},
            '127':{'name':'Honours degree', 'cost_price':0.78, 'sale_price':1.49}
}

products_df = pd.DataFrame(products)
products_df

,123,124,125,126,127
name,Hoodie,Tote bag,Pencil,T-shirt,Honours degree
cost_price,12.12,2.15,22.45,0.45,0.78
sale_price,15.0,9.99,49.99,2.99,1.49


### Joining Dataframes using Merge
What we want to do is join these datasets together. However, currently the Dataframes are both the wrong way round. We would want our rows in orders_df to be the order ID, and the rows in products_df to be the product ID. We can transform the data using the Dataframe.transpose function:

In [11]:
orders_df = orders_df.transpose()
products_df = products_df.transpose()


Transpose basically flips the axes of the Dataframe so the current columns become rows and vice versa. Note, _Dataframe.transpose_ creates a new object by default. We use the "orders_df = ..." element at the start to reassign the variable and make orders_df equal the new, transposed version.

In [12]:
orders_df

,date,product,quantity
o10001,2024/01/10,Hoodie,1
o10002,2024/01/13,Tote bag,2
o10003,2024/01/14,Pencil,10
o10004,2024/01/15,T-shirt,2


In [13]:
products_df


,name,cost_price,sale_price
123,Hoodie,12.12,15.0
124,Tote bag,2.15,9.99
125,Pencil,22.45,49.99
126,T-shirt,0.45,2.99
127,Honours degree,0.78,1.49


With this in place we can join the two Dataframes. There are several decisions that need to be taken when joining two datasets. Firstly, there are multiple ways to join data in pandas but the most appropriate here would seem to be using the _merge( )_ function. These choices will be discussed in more detail later in the notebook. Secondly we need to decide on the element that we will make the join on. In this case, the data that the two dataframes have in common are the product name, labelled “product” in orders_df and as “name” in products_df. We also need to decide the type of join – what do we want to do with rows that are in one set but not in the other? This too is discussed further below. In this case the scenario is to get more details on our orders, so we are joining products_df to orders_df so we will base our rows on the orders_df dataframe. With these choices in mind we get the following code:

In [14]:
joined_df = orders_df.merge(products_df, how='left', left_on='product', right_on='name')

We’ll unpack this a little more because it is a more complex function. We are creating a new dataframe called joined_df. This will be based on the orders_df being merged with products_df as a ‘left join’. This means we will use the rows of the orders_df not products_df (as orders_df is on the left hand of the command). Finally we need to tell Python which columns we will be joining based on. In this case the left hand dataframe (orders_df) lists the relevant data as “product”, and the right hand (products_df) lists it as “name”.

In [15]:
joined_df

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


There are several types of joins that can be used here. The main types are shown below. The yellow bit in each Venn diagram represents the bit we would keep if we perform this type of join.

So far we have only looked at a "left join" (labelled in pandas as "left"). It would also be possible to specify an "inner join" ("inner"). This means we only keep records (rows) that are in both datasets. Only the records in both would be included if we use an "inner join". Finally we could use a "full join" ("outer") which means that we keep all the records irrespective of whether they are missing in one of the sides.

![](https://drive.google.com/uc?export=view&id=1e8rPwaCqpnrB9-yIcqKOm2sb6VUPGz_g)

Let’s return to our first merged dataframe (joined_df) as this better fits the description of this problem (we care more about orders in this problem than products). One immediate issue is that we have duplicated columns product and name. Let’s tidy this up by dropping the name column:

In [16]:
joined_df = joined_df.drop(['name'], axis=1)
joined_df

,date,product,quantity,cost_price,sale_price
0,2024/01/10,Hoodie,1,12.12,15.0
1,2024/01/13,Tote bag,2,2.15,9.99
2,2024/01/14,Pencil,10,22.45,49.99
3,2024/01/15,T-shirt,2,0.45,2.99


The _drop( )_ function can be used to remove columns or rows. axis=1 indicates that we want to a drop a column(s), axis=0 would mean we were dropping a row(s). Note as well that although we are only dropping one column we still need to present the column name inside a list as this is what the function expects (a list).

### EXERCISE
In the above example we merged using a left join. How would different types of joins effect the final dataframe? Experiment with inner join, right join and outer join. What impact does it have on the data retained? Are the results what you expected them to be?

In [18]:
# Perform an outer join between orders_df and products_df
# 'how="outer"' ensures all rows from both DataFrames are included
# 'left_on="product"' specifies the column from orders_df to join on
# 'right_on="name"' specifies the column from products_df to join on
outer_joined_df = orders_df.merge(products_df, how='outer', left_on='product', right_on='name')

# Display the resulting DataFrame
display(outer_joined_df)

,date,product,quantity,name,cost_price,sale_price
0,NaN,NaN,NaN,Honours degree,0.78,1.49
1,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99
4,2024/01/13,Tote bag,2,Tote bag,2.15,9.99


In [19]:
# Perform an inner join between orders_df and products_df
# 'how="inner"' ensures only rows with matching keys in both DataFrames are included
# 'left_on="product"' specifies the column from orders_df to join on
# 'right_on="name"' specifies the column from products_df to join on
inner_joined_df = orders_df.merge(products_df, how='inner', left_on='product', right_on='name')

# Display the resulting DataFrame
display(inner_joined_df)

# --- Explanation of Inner Join ---
# An inner join combines rows from two DataFrames based on a common column or index.
# It only keeps the rows where there is a match in *both* DataFrames.
# Any rows that do not have a match in the other DataFrame are discarded.
# In essence, it returns the intersection of the two DataFrames.
# For example, in this case, products like 'Honours degree' from products_df
# that do not have an order in orders_df will not be included in the result.
# Similarly, if there were orders for products not in products_df, those would also be excluded.

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


In [20]:
# Perform a right join between orders_df and products_df
# 'how="right"' ensures all rows from the right DataFrame (products_df) are included
# 'left_on="product"' specifies the column from orders_df to join on
# 'right_on="name"' specifies the column from products_df to join on
right_joined_df = orders_df.merge(products_df, how='right', left_on='product', right_on='name')

# Display the resulting DataFrame
display(right_joined_df)

# --- Explanation of Right Join ---
# A right join (or right outer join) returns all records from the 'right' DataFrame,
# and the matched records from the 'left' DataFrame. If there is no match in the left DataFrame,
# the columns from the left DataFrame will have NaN (Not a Number) values.
# In this case, 'products_df' is the right DataFrame.
# This means the resulting DataFrame will include every product listed in 'products_df',
# and if there's no corresponding order in 'orders_df' for a product,
# the order-related columns (date, product, quantity) will show NaN.

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99
4,NaN,NaN,NaN,Honours degree,0.78,1.49


# Task
Display and explain the differences between left, inner, right, and outer joins using the `joined_df`, `inner_joined_df`, `right_joined_df`, and `outer_joined_df` DataFrames, focusing on which rows and columns are retained from the original `orders_df` and `products_df`, and then summarize the understanding of these different join types.

## Display Merged DataFrames

### Subtask:
Display the four DataFrames (`joined_df` (left join), `inner_joined_df`, `right_joined_df`, and `outer_joined_df`) sequentially with clear labels for comparison.


### `joined_df` (Left Join)

**Reasoning**:
Now I will display the `joined_df` DataFrame as the first part of the comparison.



In [21]:
display(joined_df)

,date,product,quantity,cost_price,sale_price
0,2024/01/10,Hoodie,1,12.12,15.0
1,2024/01/13,Tote bag,2,2.15,9.99
2,2024/01/14,Pencil,10,22.45,49.99
3,2024/01/15,T-shirt,2,0.45,2.99


### `inner_joined_df` (Inner Join)

**Reasoning**:
Now I will display the `inner_joined_df` DataFrame as the next part of the comparison.



In [22]:
display(inner_joined_df)

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


### `right_joined_df` (Right Join)

**Reasoning**:
Now I will display the `right_joined_df` DataFrame as the next part of the comparison.



In [23]:
display(right_joined_df)

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99
4,NaN,NaN,NaN,Honours degree,0.78,1.49


### `outer_joined_df` (Outer Join)

**Reasoning**:
Now I will display the `outer_joined_df` DataFrame as the final part of the comparison.



In [24]:
display(outer_joined_df)

,date,product,quantity,name,cost_price,sale_price
0,NaN,NaN,NaN,Honours degree,0.78,1.49
1,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99
4,2024/01/13,Tote bag,2,Tote bag,2.15,9.99


## Summary of Join Types

### Left Join (`joined_df`)
- **Rows retained**: All rows from the `orders_df` (left DataFrame) are kept.
- **Columns retained**: All columns from both `orders_df` and `products_df` are included. For rows in `orders_df` that do not have a match in `products_df`, the columns from `products_df` will have `NaN` values.
- **Observations**: In our case, `joined_df` (which was a left join) initially included the duplicate 'name' column from `products_df` and did not contain 'Honours degree' because there was no corresponding order for it in `orders_df`. After dropping the 'name' column, it effectively shows all orders with their corresponding product details. Since all products in `orders_df` had a match in `products_df`, there were no `NaN` values from the `products_df` columns.

### Inner Join (`inner_joined_df`)
- **Rows retained**: Only rows where there is a match in *both* `orders_df` and `products_df` (based on the 'product' and 'name' columns) are kept.
- **Columns retained**: All columns from both DataFrames are included.
- **Observations**: `inner_joined_df` contains only the four orders that had a matching product in `products_df`. The 'Honours degree' product from `products_df` is excluded because there is no order for it in `orders_df`. Similarly, if `orders_df` had products not present in `products_df`, those orders would also be excluded. This join effectively shows the intersection of the two DataFrames.

### Right Join (`right_joined_df`)
- **Rows retained**: All rows from the `products_df` (right DataFrame) are kept.
- **Columns retained**: All columns from both `orders_df` and `products_df` are included. For rows in `products_df` that do not have a match in `orders_df`, the columns from `orders_df` will have `NaN` values.
- **Observations**: `right_joined_df` includes all products from `products_df`, including 'Honours degree'. For 'Honours degree', the order-related columns (`date`, `product`, `quantity`) are filled with `NaN` because there is no corresponding order in `orders_df`.

### Outer Join (`outer_joined_df`)
- **Rows retained**: All rows from *both* `orders_df` and `products_df` are kept. This includes matching rows, as well as non-matching rows from either DataFrame.
- **Columns retained**: All columns from both DataFrames are included. For non-matching rows, `NaN` values will appear in the columns from the DataFrame that lacks a match.
- **Observations**: `outer_joined_df` includes all four orders and all five products. The 'Honours degree' product shows `NaN` for the order details, and if there were any orders for products not in `products_df`, those would also appear with `NaN` for product details. This join provides a comprehensive view, showing everything from both DataFrames and highlighting where matches are missing.

## Summary:

### Q&A
The task implicitly asked for an explanation and summary of the differences between left, inner, right, and outer joins.

*   **Left Join (`joined_df`)**: Retains all rows from the left DataFrame (`orders_df`) and matching rows from the right DataFrame (`products_df`). If no match is found in the right DataFrame, `NaN` values fill its columns. In this case, `joined_df` included all 4 orders, and since all had product matches, no `NaN` values from `products_df` columns were observed.

*   **Inner Join (`inner_joined_df`)**: Retains only rows where there is a match in *both* DataFrames. It showed 4 entries, representing the intersection of `orders_df` and `products_df` based on the join key, excluding `products_df`'s 'Honours degree' which had no corresponding order.

*   **Right Join (`right_joined_df`)**: Retains all rows from the right DataFrame (`products_df`) and matching rows from the left DataFrame (`orders_df`). If no match is found in the left DataFrame, `NaN` values fill its columns. `right_joined_df` included all 5 products, with `NaN` values for the `date`, `product`, and `quantity` columns for 'Honours degree', as it lacked a matching order.

*   **Outer Join (`outer_joined_df`)**: Retains all rows from *both* DataFrames, filling in `NaN` values where there are no matches in the corresponding DataFrame. This join provided a comprehensive view, including all 4 orders and all 5 products, with `NaN`s for 'Honours degree' details.

### Data Analysis Key Findings
*   The `joined_df` (left join) successfully retained all four orders from `orders_df` and their corresponding product details, with no `NaN` values from `products_df` observed, indicating all products in `orders_df` had a match.
*   The `inner_joined_df` (inner join) contained exactly four rows, showcasing only the intersection where orders and products explicitly matched, thus excluding products or orders without a counterpart.
*   The `right_joined_df` (right join) included all five products from `products_df`, demonstrating that 'Honours degree', which had no corresponding order, resulted in `NaN` values for the `date`, `product`, and `quantity` columns.
*   The `outer_joined_df` (outer join) provided a complete view, including all four orders and all five products, with `NaN` values appropriately placed for non-matching entries like the order details for 'Honours degree'.

### Insights or Next Steps
*   The choice of join type is critical and directly dictates the scope of the dataset for analysis; an inner join focuses on common data, while outer joins ensure no data from either source is lost.
*   Careful consideration of potential `NaN` values introduced by joins is necessary for subsequent data cleaning and accurate analytical results, especially when dealing with data completeness.
